# Music21 Scale Tables

In this notebook we are developing a eye-candy presentation of scale data generated by music21 and our music21 toolchain.

In [4]:
from music21 import *
from IPython.display import display, HTML, Image, Audio
import plim
import base64
#from music21.common.pathTools import relativepath as m21_jupyter_relpath

In [5]:
def circleOfFifths():
    fifths = []

    fifthsScale = scale.CyclicalScale('c4', 'p5')
    
    for p in fifthsScale.getPitches('c4', 'c11'):
        
        if key.pitchToSharps(p,'major') <= 7:
            MAJOR = str(p.simplifyEnharmonic().name)
            MINOR = str(key.Key(MAJOR, 'major').relative.tonic.simplifyEnharmonic().name).lower()

        else:
            MAJOR = str(p.getEnharmonic().simplifyEnharmonic().name)
            MINOR = str(key.Key(MAJOR, 'major').relative.tonic.simplifyEnharmonic().name).lower()
        
        fifths.append( [ MAJOR, MINOR ] )

        #fifths.append(p.simplifyEnharmonic())
        
    return fifths

In [6]:
fifths = circleOfFifths()

greekModes = [ 'ionian','dorian','phrygian',
          'lydian','mixolydian','aeolian',
          'locrian','major','minor']

## The returnScale() function

This function returns a `dict()` with the data and filename of our generated scale.

In [7]:
def returnScale(tonic='c', mode='major', ownKey=False):
    
    modes = [ 'ionian','dorian','phrygian',
              'lydian','mixolydian','aeolian',
              'locrian','major','minor']
    
    if not mode in modes: mode = 'major'
        
    if mode == 'ionian': mode = 'major'
    if mode == 'aeolian': mode = 'minor'

    tonic_pitch = pitch.Pitch(tonic)
    scale_sharps = key.pitchToSharps(tonic_pitch, mode)

    scale_func = getattr(scale, "{0}Scale".format(mode.capitalize()))
    real_scale = scale_func(tonic_pitch)
    pitches = real_scale.getPitches()

    scale_name = real_scale.name
    absolute_intervals = []
    relative_intervals = []
    myKeySig = key.KeySignature(scale_sharps)
    
    mystream = stream.Stream()
    
    if ownKey:
        mystream.append(myKeySig)

    for cur_pitch in pitches:
        mynote = note.Note(cur_pitch)

        cur_degree = real_scale.getScaleDegreeFromPitch(cur_pitch)
        past_degree_or_tonic = cur_degree-1 if cur_degree>1 else 1
        
        absolute_interval = real_scale.intervalBetweenDegrees(1,cur_degree).name
        relative_interval = real_scale.intervalBetweenDegrees(past_degree_or_tonic, cur_degree).name
        
        absolute_intervals.append([absolute_interval,'colspan="2" style="width: {}%;"'.format(100/8)])
        relative_intervals.append([relative_interval, 'colspan="8" style="width: {}%;"'.format(100/8)])
        
        mynote.addLyric(absolute_interval)
        mynote.addLyric(relative_interval)
        
        mystream.append(mynote)
        
    filename = mystream.write()
        
    relative_intervals.insert(0, ['N','colspan="8" style="width: {}%;"'.format((100/8)/2)])
    relative_intervals.append(['A', 'colspan="8" style="width: {}%;"'.format((100/8)/2)])

    return {
        'scalename': scale_name,
        'stream' : mystream,
        'filename': filename,
        'ai' : absolute_intervals,
        'ri' : relative_intervals
    }

## Creating an Array with scale data

Here we create a bidimensional array with our scale data—a list of lists.

In [8]:
#my_data = returnScale(scale_data)
def generate_table_array(scale_data):
    
    # lets embed the image
    with open(scale_data['filename'],'rb') as image_file:
        b64_image_data = base64.b64encode(image_file.read()).decode('utf8')

    imgtag  = "<img src='data:image/png;charset=utf-8;base64,{}' />".format(b64_image_data)

    my_table_data = [
        [ [scale_data['scalename'],''] ],
        [ [imgtag,'style="padding:0;"'] ],
        scale_data['ai'], # these are already lists
        scale_data['ri']
        ]
    return my_table_data

#array = [
#    [ (data, cell attributes) ]
#]

## Converting the data array to HTML

Here we use python `plim` library to help us convert the data array to a HTML table, the which we can then present on the Jupyter Notebook.

In [9]:
def array_to_htmltable(data, slim=False):
    t2_slim = """
table style='table-layout="fixed";'
"""

    for row in data:
        t2_slim += "\ttr\n"
        for column in row:
            
            if type(column) == list and len(column)>1:
                attrs = column[1]
            else: attrs = '';
                
            if type(column) == list:
                column = column[0]
                
            if len(row) == 1:
                t2_slim += "\t\tcaption {0} {1}\n".format(attrs, str(column))
            else:
                t2_slim += "\t\ttd {0} {1}\n".format(attrs, str(column))
    if not slim:
        return plim.preprocessor(t2_slim)
    else:
        return t2_slim

In [10]:
def returnScale(tonic='c', mode='major', ownKey=False):
    
    modes = [ 'ionian','dorian','phrygian',
              'lydian','mixolydian','aeolian',
              'locrian','major','minor']
    
    if not mode in modes: mode = 'major'
        
    if mode == 'ionian': mode = 'major'
    if mode == 'aeolian': mode = 'minor'

    tonic_pitch = pitch.Pitch(tonic)
    scale_sharps = key.pitchToSharps(tonic_pitch, mode)

    scale_func = getattr(scale, "{0}Scale".format(mode.capitalize()))
    real_scale = scale_func(tonic_pitch)
    pitches = real_scale.getPitches()

    scale_name = real_scale.name
    absolute_intervals = []
    relative_intervals = []
    myKeySig = key.KeySignature(scale_sharps)
    
    mystream = stream.Stream()
    
    if ownKey:
        mystream.append(myKeySig)

    for cur_pitch in pitches:
        mynote = note.Note(cur_pitch)

        cur_degree = real_scale.getScaleDegreeFromPitch(cur_pitch)
        past_degree_or_tonic = cur_degree-1 if cur_degree>1 else 1
        
        absolute_interval = real_scale.intervalBetweenDegrees(1,cur_degree).name
        relative_interval = real_scale.intervalBetweenDegrees(past_degree_or_tonic, cur_degree).name
        
        absolute_intervals.append([absolute_interval,'display="block;" style="width: {}%; float:left;"'.format(100/8)])
        relative_intervals.append([relative_interval, 'display="block;" style="width: {}%; float:left;"'.format(100/10)])
        #absolute_intervals.append([absolute_interval,])
        #relative_intervals.append([relative_interval,'colspan="8"'])
        
        mynote.addLyric(absolute_interval)
        mynote.addLyric(relative_interval)
        
        mystream.append(mynote)
        
    filename = mystream.write()
        
    spc = ['N', 'display="block;" style="width: {}%; float:left;"'.format(100/10)]
    relative_intervals.insert(0, spc)
    relative_intervals.append(spc)

    return {
        'scalename': scale_name,
        'stream' : mystream,
        'filename': filename,
        'ai' : absolute_intervals,
        'ri' : relative_intervals
    }

## Displaying the table

Then we can show the IPython.display to show the HTML table.

In [11]:
sc_data = returnScale()
sc_matrix = generate_table_array(sc_data)
sc_slim = array_to_htmltable(sc_matrix,slim=True)
sc_html = array_to_htmltable(sc_matrix)

display(HTML(sc_html))

/print sc_slim


table style='table-layout="fixed";'
	tr
		caption  C major
	tr
		caption style="padding:0;" <img src='data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAS4AAABsCAIAAABbxcjHAAAQxUlEQVR4nO2dMY/rRNfHZx+9CLZYZF8kbgHSYuciauyIksaOKChokkgUlHEkPkCcHgonFaKL01Em+w3sSDRQ2aa4zRWsnYvEIkQRr1ihRVqkvMV5d14/TuKMx/Zdb3J+xZWT9f1nPJ5jn5kzc+ZkvV4TBEEemv88dAEQBCEETRFBagKaIoLUAjRFBKkFaIoIUgvQFBGkFqApIkgtQFNEkFqApoggtaBMU4yiKI7jEgUR5HgozRR931dVtdPplCWIIEdFOabo+76u69fX14vFwrbtUjQR5Kg4KWU6uKqqQRDAsSRJURQV10SQo6KEt+JoNKJ2SAhZLpf4YkSQvJTwVpRleblcEkIEQRgOh4SQ2Wzm+34JpUOQo6HoW9H3fWqHruuapgnOKvqoSHFOTk4eugivjhJMEQ5s21ZVlRAC/87n84LKCHJUFDXFMAwJIZqm0TCGKIqEEM/zCiojyFFRTjCj3+/TY3hPooOKILkoxxSTkX0wwuSYKnLwHFWnriJKMEVBEJIfwTVNfYkgSDZFTbHZbMqynPwGBmxg8AZBEEb+J/vPvu+bpplxwmq1+umnn3Rdh49XV1cQ27i8vKRfInVgtVq9fPlSURSWky8vL09PT9955x12/Ypu92G0oiAIvv766y+//DLrpHUxVqsVIWS1WsGxJEkg63leQWWkXBzH0TSN8WRN0waDAbt48Yb0KmVfPYQQx3GyzynqoIqiKEmS67qEENM04ZWoaRo6qAiSiz0OKgu6rnueF8fxdDqFb2D6G4Ig7JQwgtpsNufzOe1SDgaDw/DvHxBd18HRQI6HE